## Feature Extraction

Given the previous analysis now we can address how we want to extract features from the data. The feature extraction will follow a pipeline divided by different feature categories:
- Statistical Features
- Texture Features

In [ ]:
import numpy as np
import pandas as pd

import random

from scipy.stats import entropy, skew, kurtosis
from skimage.filters import laplace, gaussian, sobel, threshold_otsu, gabor
from skimage.feature import canny, hog
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from skimage.measure import shannon_entropy, moments_hu, moments, label, regionprops

### Statistical Features

Statistical moments from each RGB channel and combinations:
- **First-order statistics:** mean, median, standard deviation, variance, sum, skewness, kurtosis
- **Percentile features:** 10th, 25th, 75th, 90th percentiles
- **Range and extrema:** min, max, range, interquartile range
- **Entropy:** Shannon entropy

With a total of **48 features**

In [ ]:
def statisticalFeatures(image):
	features = []
	for channel in range(3):
		ch_data = image[:, :, channel]
		features += [
			np.mean(ch_data),
			np.median(ch_data),
			np.std(ch_data),
			np.var(ch_data),
			np.sum(ch_data),
			skew(ch_data.flatten()),
            kurtosis(ch_data.flatten()),

			np.percentile(ch_data, 10),
			np.percentile(ch_data, 25),
			np.percentile(ch_data, 75),
			np.percentile(ch_data, 90),

			np.min(ch_data),
			np.max(ch_data),
			np.max(ch_data) - np.min(ch_data),
			np.percentile(ch_data, 75) - np.percentile(ch_data, 25),

			shannon_entropy(ch_data)
		]
	return features

### Texture Features

Particularly powerful for land use classification:
- **GLCM properties:** contrast, dissimilarity, homogeneity, ASM, energy, correlation

With a total of **72 features**

In [ ]:
def textureFeatures(image):
	features = []


	image_gray = rgb2gray(image)
	image_gray = ((image_gray - image_gray.min()) / (image_gray.max() - image_gray.min()) * 255).astype(np.uint8)	# Normalize to 0-255



	distances = [1, 2, 3]
	angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]

	for distance in distances:
		for angle in angles:
			
			glcm = graycomatrix(image_gray, distances=[distance], angles=[angle], levels=256, symmetric=True, normed=True)

			features += [
				graycoprops(glcm, "contrast")[0, 0],
				graycoprops(glcm, "dissimilarity")[0, 0],
				# graycoprops(glcm, "homogeneity")[0, 0],
				# graycoprops(glcm, "ASM")[0, 0],
				# graycoprops(glcm, "energy")[0, 0],
				# graycoprops(glcm, "correlation")[0, 0],
			]

	return features

### LBP Features

- **Local Binary Pattern** histograms and statistics for each RGB channel

With a total of **111 feautres**

In [ ]:
def lbpFeatures(image):

	features = []

	radius = 3
	n_points = 8 * radius

	for channel in range(3):

		ch_data = image[:, :, channel]

		lbp = local_binary_pattern(ch_data, n_points, radius, method="uniform")
		hist, _ = np.histogram(lbp.ravel(), bins=n_points + 2, range=(0, n_points + 2))

		features.extend(hist)

		features += [
			np.mean(lbp),
			np.std(lbp),
			np.var(lbp)
		]

	return features

### Gabor Features

- **Gabor filter** responses across diferent frequencies and orientations

With a total of **216 features**

In [ ]:
def gaborFeatures(image):
	features = []

	frequencies = [0.1, 0.3, 0.5]
	orientations = [0, np.pi/4, np.pi/2, 3*np.pi/4]

	for channel in range(3):
		channel_data = image[:, :, channel]

		for frequency in frequencies:
			for theta in orientations:

				real, _ = gabor(channel_data, frequency=frequency, theta=theta)

				features += [
					np.mean(real),
					np.std(real),
					np.var(real),
					np.max(real),
					np.min(real),
					shannon_entropy(np.abs(real))
				]

	return features

### Color Space Feautres

- Statistics from **HSV** and **LAB** color spaces

With a total of **30 features**

In [ ]:
def colorSpaceFeatures(image):

	features = []

	hsv = rgb2hsv(image)
	lab = rgb2lab(image)

	for channel in range(3):
		hsv_channel_data = hsv[:, :, channel]
		lab_channel_data = lab[:, :, channel]

		features += [
			# np.mean(hsv_channel_data),
			np.mean(lab_channel_data),
			# np.std(hsv_channel_data),
			np.std(lab_channel_data),
			# np.var(hsv_channel_data),
			np.var(lab_channel_data),
			skew(hsv_channel_data.flatten()),
			skew(lab_channel_data.flatten()),
			kurtosis(hsv_channel_data.flatten()),
			kurtosis(lab_channel_data.flatten())
		]

	return features

## Feature Pipeline

Once all categorical feature extraction functions are designed, it's required to add them to a complete pipeline to merge all features in a single Data Frame. 

In [ ]:
def featureExtraction(image):
	features = []

	features += statisticalFeatures(image)
	features += textureFeatures(image)
	features += lbpFeatures(image)
	# features += gaborFeatures(image)
	features += colorSpaceFeatures(image)


	return features

In [ ]:
data = []
labels = [] 
names = []

max_per_cat = 10

for cat in categories:
	print(cat)
	files = os.listdir(base_path + cat + "/")
	random.shuffle(files)
	for i, image in enumerate(files):
		if i == max_per_cat:
			break
		path = base_path + cat + "/" + image
		im = imread(path)
		data.append(featureExtraction(im))
		labels.append(cat)
		names.append(image)

X = np.array(data).astype(np.uint8)

In [ ]:
X.shape

In [ ]:
df = pd.DataFrame(X)

x = X
y = labels
image_paths = names
df["class"] = labels
df["name"] = names
df
print(df.to_string())

## ML Models

Once the Data Frame has been created with all the extracted features, we can use this data to train the classical ML models.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, roc_auc_score, auc
from sklearn.preprocessing import StandardScaler, LabelEncoder, label_binarize
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from collections import Counter

import seaborn as sns

In [ ]:
x = df.drop(columns = ["class", "name"])
y = df["class"]
image_paths = df["name"]
target_names = y.unique()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.25, stratify=y)
feature_columns = x_train.columns

In [ ]:
rf = RandomForestClassifier()
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred, target_names=target_names))
sns.heatmap(confusion_matrix(y_test,y_pred),annot=True, xticklabels=target_names, yticklabels=target_names)